In [1]:
import os
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.schema import  SystemMessage,HumanMessage,AIMessage,Document
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load API key from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

# load json files
def load_jfile(file:list):
    folder_path = './data/'
    loader = JSONLoader(
        file_path=folder_path+file,
        jq_schema='.[]',
        content_key='html',
    )
    text_spilter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=100)
    docs = text_spilter.split_documents(loader.load())
    return docs

class Chat_with_ai():
    def __init__(self,query,data=None,k=3):
        self.data = data
        self.query = query
        self.k = k
        self.messages = [
            SystemMessage(content="You are a helpful assistant for LangChain python framework."),
            SystemMessage(content="**Reply in zh-tw**"),
        ]
        # data ==none continue to chat
        self.db = 'None'
        self.transcript = 'None'
        if data != None:
            self.db = self.data_to_db(data)
            self.transcript = self.get()
    
    def data_to_db(self,data)->FAISS:
        embedding = OpenAIEmbeddings()
        db = FAISS.from_documents(data,embedding)
        return db
    
    def get(self):
        docs =self.db.similarity_search(query = self.query, k = self.k)
        # print(docs)
        docs_page_content = ' '.join([str(doc).replace('\n\n','.').replace('\n',' ') for doc in docs])
        return docs_page_content
    
    def chat(self):
        model = ChatOpenAI(
            openai_api_key=os.environ["OPENAI_API_KEY"],
            model='gpt-3.5-turbo'
        )
        self.messages.append(SystemMessage(content=self.transcript))
        self.messages.append(HumanMessage(content=self.query))
        response =model.invoke(self.messages).content
        self.messages.append(AIMessage(content=response))
        return response

In [2]:
"""
No data : LCEL 是 LangChain 框架的縮寫，代表 LangChain Execution Layer。這是 LangChain 框架的一部分，負責執行區塊鏈相關的操作和功能。


Have data:
    LCEL（LangChain Expression Language）是一種聲明式的方式，可輕鬆地組合鏈接起來。LCEL從一開始就設計支援將原型放入生產環境，無需進行任何代碼更改，從最簡單的“提示+LLM”鏈到最複雜的鏈（我們已經看到人們成功在生產環境中運行帶有數百步驟的LCEL鏈）。以下是您可能想要使用LCEL的一些原因：

    - LCEL使得從基本組件構建複雜鏈變得容易。
    - 提供統一接口：每個LCEL對象都實現了Runnable接口，該接口定義了一組常用的調用方法（invoke、batch、stream、ainvoke等）。這使得LCEL對象的鏈也可以自動支持這些調用。換句話說，每個LCEL對象的鏈本身也是一個LCEL對象。
    - 組合原語：LCEL提供了一些原語，可輕鬆組合鏈、並行化組件、添加回退、動態配置鏈內部等。

    若想繼續學習有關LCEL的知識，建議您：

    - 閱讀完整的LCEL介面，我們在此處僅部分涵蓋了介面。
    - 探索如何部分以了解LCEL提供的其他組合原語。
    - 查看烹飪書部分，看看LCEL在常見用例中的應用。建議查看的下一個用例可能是檢索增強型生成。

    這些是瞭解LCEL的下一步。
"""

'\nNo data : LCEL 是 LangChain 框架的縮寫，代表 LangChain Execution Layer。這是 LangChain 框架的一部分，負責執行區塊鏈相關的操作和功能。\n\n\nHave data:\n    LCEL（LangChain Expression Language）是一種聲明式的方式，可輕鬆地組合鏈接起來。LCEL從一開始就設計支援將原型放入生產環境，無需進行任何代碼更改，從最簡單的“提示+LLM”鏈到最複雜的鏈（我們已經看到人們成功在生產環境中運行帶有數百步驟的LCEL鏈）。以下是您可能想要使用LCEL的一些原因：\n\n    - LCEL使得從基本組件構建複雜鏈變得容易。\n    - 提供統一接口：每個LCEL對象都實現了Runnable接口，該接口定義了一組常用的調用方法（invoke、batch、stream、ainvoke等）。這使得LCEL對象的鏈也可以自動支持這些調用。換句話說，每個LCEL對象的鏈本身也是一個LCEL對象。\n    - 組合原語：LCEL提供了一些原語，可輕鬆組合鏈、並行化組件、添加回退、動態配置鏈內部等。\n\n    若想繼續學習有關LCEL的知識，建議您：\n\n    - 閱讀完整的LCEL介面，我們在此處僅部分涵蓋了介面。\n    - 探索如何部分以了解LCEL提供的其他組合原語。\n    - 查看烹飪書部分，看看LCEL在常見用例中的應用。建議查看的下一個用例可能是檢索增強型生成。\n\n    這些是瞭解LCEL的下一步。\n'

In [3]:
# files =['langchain_intro.json','langchain_express.json']
file = 'langchain_express.json'
j_data = load_jfile(file)
# pprint(j_data)
query = 'what is LCEL?'
model = Chat_with_ai(query,j_data)
print(model.chat())

LCEL（LangChain Expression Language）是一種聲明式的方式，可以輕鬆地組合鏈條。LCEL從一開始就設計成支持將原型放入生產環境，無需進行任何代碼更改，從最簡單的“提示+LLM”鏈條到最複雜的鏈條（我們已經看到有人成功地在生產環境中運行具有數百步驟的LCEL鏈條）。這裡列出一些您可能想要使用LCEL的原因：

- LCEL使得從基本組件構建複雜鏈條變得容易。它通過提供以下功能來實現這一點：
    1. 統一接口：每個LCEL對象都實現了Runnable接口，該接口定義了一組共同的調用方法（調用、批量、流、ainvoke等）。這使得LCEL對象的鏈條也可以自動支持這些調用。也就是說，每個LCEL對象的鏈條本身也是一個LCEL對象。
    2. 組合原語：LCEL提供了許多原語，使得輕鬆組合鏈條、並行化組件、添加回退、動態配置鏈條內部等成為可能。


In [4]:
model.query = '您如何選擇LLM進行finetune的模型和參數？'
print(model.chat())

在選擇LLM進行finetune的模型和參數時，您可以考慮以下因素：

1. 模型性能：選擇具有良好性能和適用於您特定任務的LLM模型。根據您的需求，可以考慮不同的LLM變體，如GPT-3、BERT、RoBERTa等。

2. 訓練數據：確保您的訓練數據集合適合您的任務。請確保數據質量良好，標註準確，並覆蓋您希望模型進行finetune的領域。

3. 訓練參數：調整學習率、批量大小、訓練輪數等參數以達到最佳性能。使用適當的訓練參數可以幫助模型更好地學習特定任務。

4. 驗證和調整：在訓練過程中，定期驗證模型的性能，根據驗證結果調整模型參數，以達到更好的效果。

5. 預訓練權重：如果您有先前的預訓練模型權重，可以考慮將其用作LLM模型finetune的起點，這有助於提高模型的收斂速度。

進行模型和參數選擇時，建議進行一些實驗和調整，以找到最適合您特定任務的配置。


In [5]:
model.query = '我的第一個問題是什麼'
print(model.chat())

您的第一個問題是關於LCEL（LangChain Expression Language）是什麼。


In [ ]:
"""
1. LCEL 如何提供最佳的時間到首個令牌的支持？這對於使用者有什麼具體的影響？
2. 文章提到 LCEL 可以支援異步和同步 API，您認為這樣的設計有哪些優勢？在實際應用中可能會遇到哪些情況？
3. 文章中提到 LCEL 可以自動執行具有可以並行執行步驟的鏈結，您可以提供一個示例來說明這種自動化的執行對於提高效率和降低延遲的作用嗎？
"""

In [6]:
model.query = '文章提到 LCEL 可以支援異步和同步 API，您認為這樣的設計有哪些優勢？在實際應用中可能會遇到哪些情況？'
print(model.chat())

設計支援異步和同步API的LCEL具有以下優勢：

1. 靈活性：異步和同步API的支援使得LCEL在處理不同類型的任務時更加靈活。開發人員可以根據任務的特性和需求選擇適合的API方式。

2. 效率：異步API可以提高系統的效率，特別是在處理大量請求或需要長時間處理的任務時。同步API則可以確保任務的連貫性和順序執行。

3. 高性能：異步API可以讓系統同時處理多個任務，提高系統的並行處理能力，從而提升整體性能。

在實際應用中，可能遇到以下情況：

1. 高併發處理：當系統需要同時處理大量請求時，使用異步API可以提高系統的併發處理能力，確保系統的效率和性能。

2. 非同步任務：某些任務可能需要長時間執行，如大規模數據處理或模型訓練。使用異步API可以避免阻塞主線程，使系統可以同時執行其他任務。

3. 事件驅動架構：在事件驅動的應用中，異步API可以方便地處理事件的觸發和處理，確保系統的即時性和反應速度。

總的來說，異步和同步API的支援使得LCEL在設計和應用時更具彈性和效率，能夠應對不同的需求和情況，提升系統整體性能和用戶體驗。


In [7]:
model.query = '文章中提到 LCEL 可以自動執行具有可以並行執行步驟的鏈結，您可以提供一個示例來說明這種自動化的執行對於提高效率和降低延遲的作用嗎？'
print(model.chat())

當LCEL自動執行具有可以並行執行步驟的鏈結時，可以提高效率和降低延遲的作用。以下是一個示例來說明這種自動化執行的優勢：

假設有一個複雜的處理流程，其中包含多個步驟需要依次執行，並且某些步驟可以並行執行。這個處理流程可以用LCEL來建模和自動執行。

1. 步驟1：從資料庫中檢索資料。
2. 步驟2：對檢索到的資料進行預處理。
3. 步驟3：將預處理後的資料輸入模型進行訓練。
4. 步驟4：將訓練後的模型應用於新的資料進行預測。
5. 步驟5：將預測結果存儲回資料庫。

在這個例子中，步驟1和步驟2是可以並行執行的，因為它們之間沒有依賴關係。利用LCEL自動化執行，系統可以同時執行這兩個步驟，從而節省時間並提高效率。

這種並行執行可以降低整個處理流程的延遲，因為系統在執行不需要依賴的步驟時不需要等待前一個步驟完成。同時，自動化執行也可以確保步驟的順序和一致性，減少人為錯誤的風險，提高系統的可靠性和穩定性。

因此，通過LCEL自動執行具有並行執行步驟的鏈結，可以有效提高效率、降低延遲，並簡化複雜流程的管理和執行。


In [12]:
model.query = '我的上一個問題是什麼'
print(model.chat())

您上一個問題是關於LCEL支援異步和同步API的設計，以及自動執行具有可以並行執行步驟的鏈結。您問到這種自動化執行對於提高效率和降低延遲的作用。希望這個回答對您有幫助。如果您有任何其他問題，請隨時告訴我。


In [13]:
model.query = '文章中提到 LCEL 可以自動執行具有可以並行執行步驟的鏈結，您可以提供一個示例來說明這種自動化的執行對於提高效率和降低延遲的作用嗎？'
print(model.chat())

當LCEL自動執行具有可以並行執行步驟的鏈結時，可以提高效率和降低延遲。以下是一個示例來說明這種自動化執行的作用：

假設有一個複雜的處理流程，其中包含以下步驟：
1. 步驟1：從資料庫中檢索資料。
2. 步驟2：對檢索到的資料進行預處理。
3. 步驟3：將預處理後的資料輸入多個模型進行訓練。
4. 步驟4：將這些模型應用於新的資料進行預測。
5. 步驟5：將預測結果存儲回資料庫。

在這個例子中，步驟1和步驟2是可以同時執行的，因為它們之間沒有依賴關係。通過LCEL的自動化執行，系統可以同時執行這兩個步驟，從而節省時間並提高效率。

這種並行執行可以降低整個處理流程的延遲，因為系統在執行不需要依賴的步驟時不需要等待前一個步驟完成。同時，自動化執行也可以確保步驟的順序和一致性，減少人為錯誤的風險，提高系統的可靠性和穩定性。

因此，通過LCEL自動執行具有可以並行執行步驟的鏈結，可以有效提高效率、降低延遲，並簡化複雜流程的管理和執行。
